# Custom Action Features

## Overview

### Questions

- How do I access simulation state information?
- How do I create loggable quantities in custom actions?
- What are other features provided by the custom action/operation
  API?

### Objectives

- Explain how to access simulation state in a custom action.
- Explain how to expose loggable quantities in a custom action.
- Demonstrate other miscellaneous features fo custom actions.

## Boilerplate Code

In [1]:
import hoomd

def clear_sim(sim):
    """Remove all operations (besides intergrator from the sim."""
    sim.operations.updaters.clear()
    sim.operations.writers.clear()
    del sim.operations.tuners[1:] # keep the particle sorter

cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu)

snap = hoomd.Snapshot()
snap.particles.N = 1
snap.particles.position[:] = [0, 0, 0]
snap.particles.types = ['A']
snap.particles.typeid[:] = [0]
snap.configuration.box = [10, 10, 10, 0, 0, 0]

sim.create_state_from_snapshot(snap)

## How do I access simulation state?

By the time that a custom action will have its `act` method called
it will have an attribute `_state` accessable to it which is the
simulation state for the simulation it is associated with. The behavior
of this is controlled in the `hoomd.custom.Action.attach` method. The
method takes in a simulation object and performs any necessary set-up
for the action call `act`. By default, the method stores the simulation
state in the `_state` attribute.

We will create two custom actions class to show this. In one, we will
not modify the `attach` method, and in the other we will make `attach`
a no-op. In each class we will make the action test for the existance
of the `_state` attribute and print some information about the `_state`
if it exists.

In [2]:
class InspectState(hoomd.custom.Action):
    def act(self, timestep):
        if hasattr(self, '_state'):
            print("Has state.")
            print(f"Type {type(self._state)}, ", end='')
            print(f"Number of particles {self._state.N_particles}.")
        else:
            print("Uh-oh no '_state' to be found.")

class BrokenInspectState(InspectState):
    def attach(self, simulation):
        pass

Like in the previous section these are both writers. We will go ahead
and wrap them and see what happens when we try to run the simulation.

In [3]:
inspect_state = InspectState()
sim.operations.writers.append(
    hoomd.write.CustomWriter(inspect_state,
                             trigger=hoomd.trigger.Periodic(10))
)
sim.run(11)

Has state.
Type <class 'hoomd.state.State'>, Number of particles 1.


In [4]:
clear_sim(sim)
broken_inspect_state = BrokenInspectState()
sim.operations.writers.append(
    hoomd.write.CustomWriter(broken_inspect_state,
                             trigger=hoomd.trigger.Periodic(10))
)
sim.run(11)

Uh-oh no _state to be found.
